In [60]:
import bibtexparser
from iso4 import abbreviate
import nltk

import re

In [61]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/inon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
def _abrv(full):
    return re.sub(pattern=r'\s+', 
                  repl=' ', 
                  string=abbreviate(full))

# Example
_abrv('Accident Analysis \\& Prevention')

'Accid. Anal. Prev.'

In [63]:
def _to_bibkey(full):
    return ''.join(
        filter(str.isalnum,
               map(lambda word: word[0].upper(), _abrv(full=full).split(' '))
              )
    )

# Example
_to_bibkey(full='Accident Analysis \\& Prevention')

'AAP'

In [64]:
def _get_journals(bibtex_references_as_string):
    return frozenset(map(
        lambda entry: entry['journal'], 
        filter(lambda entry: 'journal' in entry, 
               bibtexparser.loads(bibtex_references_as_string).entries)))


# Example
list(
    _get_journals(bibtex_references_as_string=open('ref.bib').read())
)[:3]

['Accident Analysis \\& Prevention',
 'Water resources research',
 'Journal of the Royal Statistical Society: Series B (Methodological)']

In [69]:
def _journals_to_bib(bibtex_references_as_string):
    return dict(map(
        lambda full: (full, {'bibkey': _to_bibkey(full), 'abrv': _abrv(full)}),
        _get_journals(bibtex_references_as_string)
        )
    )

# Example
list(
    _journals_to_bib(bibtex_references_as_string=open('ref.bib').read()).items()
)[:3]

[('Accident Analysis \\& Prevention',
  {'bibkey': 'AAP', 'abrv': 'Accid. Anal. Prev.'}),
 ('Water resources research', {'bibkey': 'WRR', 'abrv': 'Water resour. res.'}),
 ('Journal of the Royal Statistical Society: Series B (Methodological)',
  {'bibkey': 'JRSSSB', 'abrv': 'J. R. Stat. Soc. Ser. B (methodological'})]

In [115]:
def _to_bibstring(bibkey, value):
    return '@STRING{%s = "%s"}' % (bibkey, value)

In [117]:
def iso4abrv(bibtex_references_as_string, out_dir):
    parsed = bibtexparser.loads(bibtex_references_as_string)
    mapping = _journals_to_bib(bibtex_references_as_string=bibtex_references_as_string)
    for entry in filter(lambda entry: 'journal' in entry, parsed.entries):
        entry['journal'] = mapping[entry['journal']]['bibkey']
    abbreviated = bibtexparser.dumps(parsed)
    with open('abbrvref.bib', 'w') as outref_f,\
        open('jourabrv.bib', 'w') as outabrv_f,\
        open('jourfull.bib', 'w') as outfull_f:
            outref_f.write(
                re.sub(pattern=r'journal\s+=\s+[{]([^}]*)[}]', 
                       repl=r'journal = \1', 
                       string=abbreviated))
            outabrv_f.write('\n'.join(map(
                lambda full: _to_bibstring(mapping[full]['bibkey'], mapping[full]['abrv']),
                mapping
            )))
            outfull_f.write('\n'.join(map(
                lambda full: _to_bibstring(mapping[full]['bibkey'], full),
                mapping
            )))            


iso4abrv(bibtex_references_as_string=open('ref.bib').read(),
         out_dir='.')